- Get network for a bounding box
- Bring in flow data
- Measure centrality
- Derive edge-level attractiveness

In [ ]:
-0.102310,51.464500,-0.006866,51.504897

In [1]:
import OSM2AT

c:\Users\EVINTER\.conda\envs\osm_data\Lib\site-packages\OSM2AT\get_at_networks.py:31: FutureWarning: The `utils.config` function is deprecated and will be removed in the v2.0.0 release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  ox.config(use_cache=True, log_console=True, useful_tags_way=utw)


In [2]:
bounding_box = tuple([-0.102310,51.464500,-0.006866,51.504897])
G,edges = OSM2AT.get_cycle_network(bounding_box,'mode-rule','ottawa','bb',None)

c:\Users\EVINTER\.conda\envs\osm_data\Lib\site-packages\OSM2AT\get_at_networks.py:48: FutureWarning: The `north`, `south`, `east`, and `west` parameters are deprecated and will be removed in the v2.0.0 release. Use the `bbox` parameter instead.
  G = ox.graph_from_bbox(bounding_box[3],bounding_box[1], bounding_box[0], bounding_box[2],network_type = 'bike',retain_all=True,simplify=False)


UnboundLocalError: cannot access local variable 'response' where it is not associated with a value